# Présentation

Ecole : Ecole natioanle de la Statistique et de l'Analyse économique ENSAE Pierre NDIAYE de Dakar 
Classe : ISE1-Cycle long
Année académique : 2024-2025
Module : Statistique exploratoire spacial  avec M.Aboubacre HEMA       
Devoir de maison de la séance du vendredi 8 novembre 2024 
Pays : Burkina Faso  Groupe : Logicils e(TPs 1 and 2)/ GEE (TPs 3 and 4)/ Pytho l(TPs 5 and 6                                                           #   Composé de : Ange Emilson Rayan RAHERINASOLO, Khadidiatou DIAKHATElioune    Abdou Salam KANE et Awa DIAW 

# Consigne

### TP 6 ###

1. Importer les points
2. Compter le nombre de points par niveau administratif 
3. A partir des points, créer un raster de 5km par pixel (regrouper les points)
4. Visualiser les rasters sous forme de catégories

# Solution

In [38]:
# Nous commençons par importer toutes les library nécessaires (7)


import pandas as pd  # gestion des données
import geopandas as gpd  # manipuler des données géospatiales
import rasterio  # lecture et écriture de fichiers raster
from rasterio.transform import from_origin  # fonction pour créer une transformation affine pour les rasters
import matplotlib.pyplot as plt  # visualisation graphique
import numpy as np  # pour le calcul numérique
from shapely.geometry import Point  # travailler avec des objets géométriques de type point
import os  # interagir avec le système de fichiers (chemins, fichiers, etc.)
import folium  # cartes interactives 
from folium import plugins  # plugins supplémentaires de folium pour enrichir les cartes
from folium import raster_layers  # ajouter des couches raster à une carte folium
from branca.colormap import linear  # pour créer des palettes de couleurs linéaires dans folium

In [52]:
# 1. Importer les points contenus dans le fichier Points_data.csv et filtrer uniquement le pays qui nous intéresse : le Burkina Faso


path = r"C:\Users\hp\Desktop\AwaDIAW_ISE-CL\Semestre1\Informatique_Statistique\Statistique_exploratoire\TP6" #Chemin relatif
file = "Points_data.csv"
data = pd.read_csv(os.path.join(path, file))
burkina_points = data[data['event_id_cnty'].str.startswith('BFO')] # Filtre
print(f"Le nombre de points du Burkina Faso : {len(burkina_points)}") # Checking
print(burkina_points.head(2)) # 2 lignes suffisent comme il y a beaucoup de colonnes

Le nombre de points du Burkina Faso : 11392
  event_id_cnty       event_date  year  time_precision       disorder_type  \
1      BFO12659  04 October 2024  2024               1  Political violence   
2      BFO12661  04 October 2024  2024               1  Political violence   

                   event_type    sub_event_type  \
1  Explosions/Remote violence  Air/drone strike   
2                     Battles       Armed clash   

                                         actor1 assoc_actor_1        inter1  \
1       Military Forces of Burkina Faso (2022-)           NaN  State forces   
2  JNIM: Group for Support of Islam and Muslims           NaN   Rebel group   

   ...     location latitude longitude geo_precision                source  \
1  ...  Dida Forest  10.0116   -4.0287             2    Undisclosed Source   
2  ...        Djibo  14.0875   -1.6418             2  Al Zallaqa; Whatsapp   

          source_scale                                              notes  \
1  Local partner-

In [53]:
# 2. Compter le nombre de points par niveau admin (pays, région, provine et commune)


# Comptage
count_pays = burkina_points['country'].value_counts() #country = pays (admin0 lors des TP précédents)
count_region = burkina_points['admin1'].value_counts() #admin1=région
count_departement = burkina_points['admin2'].value_counts() #admin2=province ou département
count_commune = burkina_points['admin3'].value_counts() # admin3=district ou commune

# Affichage 
print("Nombre de points du pays :\n", count_pays)
print("Nombre de points par région :\n", count_region)
print("\nNombre de points par département :\n", count_departement)
print("\nNombre de points par commune :\n", count_commune)

#Exportation sous .csv


Nombre de points du pays :
 country
Burkina Faso    11392
Name: count, dtype: int64
Nombre de points par région :
 admin1
Sahel                2811
Est                  2195
Centre-Nord          1489
Boucle du Mouhoun    1165
Nord                 1115
Centre                699
Centre-Est            588
Hauts-Bassins         435
Cascades              354
Sud-Ouest             225
Centre-Ouest          190
Centre-Sud             84
Plateau-Central        42
Name: count, dtype: int64

Nombre de points par département :
 admin2
Soum           1057
Oudalan         783
Gourma          721
Sanmatenga      705
Kadiogo         699
Tapoa           607
Seno            602
Yatenga         599
Bam             451
Kossi           408
Yagha           369
Loroum          348
Koulpelogo      341
Comoe           340
Namentenga      333
Kompienga       324
Gnagna          307
Sourou          305
Houet           255
Komandjari      236
Boulgou         211
Poni            160
Mouhoun         155
Nayala    

In [54]:
#### G O T T A  C H E C K

total_pays = count_pays.sum() # Calcul des totaux
total_region = count_region.sum()
total_departement = count_departement.sum()
total_commune = count_commune.sum()

if total_pays == total_region == total_departement == total_commune: #Vérification
    print("\nLes sommes des points par commune, département, région et pays sont bien égales et sont égales à 11 392 points.")
else:
    print("\n Désolée, il y a eu incohérence dans les sommes des points :")
    if total_pays != total_region:
        print(f" - Différence entre pays ({total_pays}) et région ({total_region})")
    if total_region != total_departement:
        print(f" - Différence entre région ({total_region}) et département ({total_departement})")
    if total_departement != total_commune:
        print(f" - Différence entre département ({total_departement}) et commune ({total_commune})")



Les sommes des points par commune, département, région et pays sont bien égales et sont égales à 11 392 points.


In [62]:
#3. A partir des points, créer un raster de 5km par pixel (regrouper les points) // En 5 étapes


geometry = [Point(xy) for xy in zip(burkina_points['longitude'], burkina_points['latitude'])] # First step : on convertit les points en GeoDataFrame
gdf = gpd.GeoDataFrame(burkina_points, geometry=geometry)

#Etape2 : les paramètres du raster
pixel_size = 5000/111320  # 5 km =5000 m converti en degré vue que les x et y sont en degrés. A l'inverse, on aurait pu convertir ces derniers en m
xmin, ymin, xmax, ymax = gdf.total_bounds  # on cherche à encadrer le pays en récupérant, les coordonnées maximales
width = int((xmax - xmin) / pixel_size)  # largeur du raster
height = int((ymax - ymin) / pixel_size)  # hauteur du raster
transform = from_origin(xmin, ymax, pixel_size, pixel_size)  # La fonction from_origin() génère une transformation affine qui permet de convertir les coordonnées géographiques (X, Y) en coordonnées raster (index de ligne et de colonne). 
#En d'autres termes, elle définit la relation entre les coordonnées géographiques de l'image et l'index des pixels dans l'image.
print(width)
print(height)

raster_data = np.zeros((height, width), dtype=int) # Etape3 : raster vide

for point in gdf.geometry: # 4th step : Remplissage de notre raster
    x, y = ~transform * (point.x, point.y) #Transformation inverse (coordonnées rasters e coordonnées géographiques)
    row, col = int(y), int(x)  # les indices de position des rasters // donc forcément en entier 
    if 0 <= row < height and 0 <= col < width:  # Check if the point falls within raster bounds
        raster_data[row, col] += 1  # Incréméntation

with rasterio.open(  #Etape 5 : Enregistrement du raster
    'burkina_tous_points_raster.tif', 'w',
    driver='GTiff',
    height=height,
    width=width,
    count=1,
    dtype=raster_data.dtype,
    crs="EPSG:4326",
    transform=transform
) as dst:
    dst.write(raster_data, 1)

print("Le raster a été créé avec succès sous le nom 'burkina_tous_points_raster.tif'")
raster_path = 'burkina_tous_points_raster.tif' #Vérification du contenu de notre raster
with rasterio.open(raster_path) as src:
    # Lire la première bande
    bandeun = src.read(1)  # 1 pour la première bande
np.unique(bandeun)

208
297
Le raster a été créé avec succès sous le nom 'burkina_tous_points_raster.tif'


array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  44,  46,  47,  48,  49,  51,  52,  53,  54,
        58,  59,  60,  61,  62,  65,  66,  70,  81, 110, 116, 138, 152,
       217, 639])

In [64]:
#4. Visualiser les rasters sous forme de catégories


raster_path = 'burkina_points_raster.tif' 
with rasterio.open(raster_path) as src:
    # Lire la première bande
    first_band = src.read(1)

xmin, ymin, xmax, ymax = src.bounds 
center = [(ymin + ymax) / 2, (xmin + xmax) / 2]

m = folium.Map(location=center, zoom_start=10) # la carte avec Folium, centrée sur les coordonnées du raster

min_value, max_value = np.min(first_band), np.max(first_band) # palette de couleurs en fonction de la distribution des valeurs uniques
colormap = linear.Reds_09.scale(min_value, max_value)  # Palette de couleurs pour les valeurs

raster_layer = raster_layers.ImageOverlay(# Ajouter le raster à la carte en tant que couche d'image 
    image=first_band,  
    bounds=[[ymin, xmin], [ymax, xmax]],
    opacity=0.7,
    colormap=colormap
)

raster_layer.add_to(m) # Ajouter la couche

colormap.add_to(m) # Légende

folium.LatLngPopup().add_to(m)  # Afficher les coordonnées de la souris
plugins.Fullscreen().add_to(m)  # Basculement en mode plein écran
plugins.MeasureControl(primaryLengthUnit="kilometers").add_to(m)  # Mesure des distances

m.save(path+'interactive_map.html') # Afficher la carte
print("Bravo, la carte interactive a été enregistrée sous 'interactive_map.html'.")


Bravo, la carte interactive a été enregistrée sous 'interactive_map.html'.


# Fin du TP6